In [69]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from typing import Counter
import pandas as pd
import numpy as np

In [70]:
# Carregar e limpar os dados
file_path = './dataset/PCOS_data.csv'
data = pd.read_csv(file_path)

data_cleaned = data.replace(',', '.', regex=True)

# Convertendo colunas para os tipos numéricos apropriados
for col in data_cleaned.columns:
    try:
        data_cleaned[col] = pd.to_numeric(data_cleaned[col])
    except ValueError:
        pass

# Remover duplicatas
duplicates = data_cleaned.duplicated().sum()
data_cleaned = data_cleaned.drop_duplicates()

# Verificar valores de cada classe para balanceamento
class_distribution = data_cleaned['PCOS (Y/N)'].value_counts()

class_distribution

PCOS (Y/N)
0    364
1    177
Name: count, dtype: int64

In [71]:
# Separar características (X) e rótulos (y)
X = data_cleaned.drop(columns=['PCOS (Y/N)'])
y = data_cleaned['PCOS (Y/N)']

# Balanceamento dos dados com SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

print(f"Distribuição das classes após SMOTE: {Counter(y_balanced)}")

Distribuição das classes após SMOTE: Counter({0: 364, 1: 364})


In [72]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Treinando o modelo Random Forest com todas as características
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train_scaled, y_train)

# Predição com todas as características
y_pred_all_features_rf = random_forest.predict(X_test_scaled)

# Validação cruzada com todas as características
cv_scores_all_features_rf = cross_val_score(random_forest, X_train_scaled, y_train, cv=5, scoring='accuracy')
mean_accuracy_all_rf = np.mean(cv_scores_all_features_rf)
std_accuracy_all_rf = np.std(cv_scores_all_features_rf)

print("Relatório de Classificação com todas as características (Random Forest):\n", classification_report(y_test, y_pred_all_features_rf))
print("Matriz de Confusão com todas as características (Random Forest):\n", confusion_matrix(y_test, y_pred_all_features_rf))
print(f"Validação cruzada com todas as características (Random Forest):\nAcurácia média: {mean_accuracy_all_rf:.4f}, Desvio padrão: {std_accuracy_all_rf:.4f}")

Relatório de Classificação com todas as características (Random Forest):
               precision    recall  f1-score   support

           0       0.91      0.87      0.89       110
           1       0.88      0.92      0.90       109

    accuracy                           0.89       219
   macro avg       0.90      0.90      0.89       219
weighted avg       0.90      0.89      0.89       219

Matriz de Confusão com todas as características (Random Forest):
 [[ 96  14]
 [  9 100]]
Validação cruzada com todas as características (Random Forest):
Acurácia média: 0.9036, Desvio padrão: 0.0453


In [73]:
from joblib import dump
dump(random_forest, 'randomForest_allFeatures.joblib')

['randomForest_allFeatures.joblib']

### Treinando com as K Melhores

In [74]:
# Seleção de características com SelectKBest
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]

print("Características selecionadas:", selected_features)

Características selecionadas: Index(['Weight (Kg)', 'Cycle(R/I)', 'AMH(ng/mL)', 'Weight gain(Y/N)',
       'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Pimples(Y/N)',
       'Fast food (Y/N)', 'Follicle No. (L)', 'Follicle No. (R)'],
      dtype='object')


In [75]:
# Balanceamento dos dados com SMOTE para as características selecionadas
X_balanced_selected, y_balanced_selected = smote.fit_resample(X_new, y)

# Divisão em treino e teste para as características selecionadas
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_balanced_selected, y_balanced_selected, test_size=0.3, random_state=42)

# Normalizando os dados
X_train_scaled_selected = scaler.fit_transform(X_train_selected)
X_test_scaled_selected = scaler.transform(X_test_selected)

In [76]:
# Treinando o modelo Random Forest com as melhores características
random_forest.fit(X_train_scaled_selected, y_train_selected)

# Predição com as melhores características
y_pred_selected_rf = random_forest.predict(X_test_scaled_selected)

# Validação cruzada com as melhores características
cv_scores_selected_features_rf = cross_val_score(random_forest, X_train_scaled_selected, y_train_selected, cv=5, scoring='accuracy')
mean_accuracy_selected_rf = np.mean(cv_scores_selected_features_rf)
std_accuracy_selected_rf = np.std(cv_scores_selected_features_rf)

print("Relatório de Classificação com as melhores características (Random Forest):\n", classification_report(y_test_selected, y_pred_selected_rf))
print("Matriz de Confusão com as melhores características (Random Forest):\n", confusion_matrix(y_test_selected, y_pred_selected_rf))
print(f"Validação cruzada com as melhores características (Random Forest):\nAcurácia média: {mean_accuracy_selected_rf:.4f}, Desvio padrão: {std_accuracy_selected_rf:.4f}")

Relatório de Classificação com as melhores características (Random Forest):
               precision    recall  f1-score   support

           0       0.93      0.93      0.93       110
           1       0.93      0.93      0.93       109

    accuracy                           0.93       219
   macro avg       0.93      0.93      0.93       219
weighted avg       0.93      0.93      0.93       219

Matriz de Confusão com as melhores características (Random Forest):
 [[102   8]
 [  8 101]]
Validação cruzada com as melhores características (Random Forest):
Acurácia média: 0.9135, Desvio padrão: 0.0202


In [77]:
from joblib import dump
dump(random_forest, 'randomForest_bestFeatures.joblib')

['randomForest_bestFeatures.joblib']